In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import pickle

In [2]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
le=LabelEncoder()
data['Gender']=le.fit_transform(data['Gender'])


In [5]:
from sklearn.preprocessing import OneHotEncoder
he = OneHotEncoder(sparse_output=False, dtype=int)
he_g=he.fit_transform(data[['Geography']])
he_g


array([[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       ...,
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [6]:
code_df=pd.DataFrame(he_g,columns=he.get_feature_names_out(['Geography']))

In [7]:
#combine

data=pd.concat([data.drop('Geography',axis=1),code_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [8]:
##save encoders and scaler

with open('le.pkl','wb') as file:
    pickle.dump(le,file)
    
with open('he.pkl','wb') as file:
    pickle.dump(he,file)
    


In [9]:
X=data.drop('Exited',axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=20,random_state=42)

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [10]:
with open('sc.pkl', 'wb') as file:
    pickle.dump(sc, file)

In [11]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [12]:
#build  model
model=Sequential([Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #no.of neurons in hidden layers is 64 HL1
                  Dense(32,activation='relu'),#hl2
                  Dense(1,activation='sigmoid') #o/p layer
                  ]) 



In [13]:
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [14]:
##compile model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
#here adam has fixed learningrate

In [15]:
#set up tensorboard
from tensorflow.keras.callbacks import EarlyStopping
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [16]:
##sset earlystopping this is bcoz 
early=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
#patience is it checks for 5 epochs if value doesnt change then it stops

In [17]:
##train model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early])

Epoch 1/100


312/312 [==============================] - 3s 4ms/step - loss: 0.4414 - accuracy: 0.8103 - val_loss: 0.4791 - val_accuracy: 0.8000
Epoch 2/100
312/312 [==============================] - 1s 3ms/step - loss: 0.3708 - accuracy: 0.8489 - val_loss: 0.3900 - val_accuracy: 0.8500
Epoch 3/100
312/312 [==============================] - 1s 2ms/step - loss: 0.3472 - accuracy: 0.8598 - val_loss: 0.3597 - val_accuracy: 0.8000
Epoch 4/100
312/312 [==============================] - 1s 3ms/step - loss: 0.3404 - accuracy: 0.8603 - val_loss: 0.3257 - val_accuracy: 0.8500
Epoch 5/100
312/312 [==============================] - 1s 3ms/step - loss: 0.3346 - accuracy: 0.8621 - val_loss: 0.3148 - val_accuracy: 0.8500
Epoch 6/100
312/312 [==============================] - 1s 3ms/step - loss: 0.3313 - accuracy: 0.8632 - val_loss: 0.2964 - val_accuracy: 0.9000
Epoch 7/100
312/312 [==============================] - 1s 3ms/step - loss: 0.3284 - accuracy: 0.8643 - val_loss: 0.2937 - val_accuracy: 0.85

In [18]:
#load tensor board extension
%load_ext tensorboard

In [19]:
%tensorboard --logdir logs/fit20250812-191415

In [20]:
model.save('my_model.h5')

c:\Users\nambu\anaconda3\envs\venv4\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
